# Caffe2 GPU Distributed


## Introduction

This example demonstrates how to run standard Caffe2 [resnet50_trainer.py](https://github.com/caffe2/caffe2/blob/master/caffe2/python/examples/resnet50_trainer.py) example using Batch AI. You can run it on a single or multiple compute nodes.

## Details

- Standard Caffe2 sample script [resnet50_trainer.py](https://github.com/caffe2/caffe2/blob/master/caffe2/python/examples/resnet50_trainer.py) is used;
- MNIST Dataset has been translated into a lmdb database, and can be obtained at http://download.caffe2.ai/databases/mnist-lmdb.zip;
- Automatically created NFS folder will be used for rendezvous temp files to coordinate between each shard/node 
- Standard output of the job will be stored on Azure File Share.

## Instructions

### Install Dependencies and Create Configuration file.
Follow [instructions](/recipes) to install all dependencies and create configuration file.

### Read Configuration and Create Batch AI client

In [ ]:
from __future__ import print_function

from datetime import datetime
import os
import sys
import zipfile

from azure.storage.file import FileService
import azure.mgmt.batchai.models as models

# utilities.py contains helper functions used by different notebooks
sys.path.append('../../')
import utilities

cfg = utilities.Configuration('../../configuration.json')
client = utilities.create_batchai_client(cfg)
utilities.create_resource_group(cfg)

### Create File Share

For this example we will create a new File Share with name `batchaicntkgpupythonsample` under your storage account.

**Note** You don't need to create new file share for every cluster. We are doing this in this sample to simplify resource management for you.

In [ ]:
azure_file_share_name = 'batchaisample'
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_share(azure_file_share_name, fail_on_exist=False)
print('Done')

### Configure Compute Cluster

- For this example we will use a GPU cluster of `STANDARD_NC6` nodes. Number of nodes in the cluster is configured with `nodes_count` variable;
- We will mount file share at folder with name `afs`. Full path of this folder on a computer node will be `$AZ_BATCHAI_MOUNT_ROOT/afs`;
- We will call the cluster `nc6`;

So, the cluster will have the following parameters:

In [ ]:
azure_file_share = 'afs'
nodes_count = 2
cluster_name = 'nc6'

volumes = models.MountVolumes(
    azure_file_shares=[
        models.AzureFileShareReference(
            account_name=cfg.storage_account_name,
            credentials=models.AzureStorageCredentialsInfo(
                account_key=cfg.storage_account_key),
            azure_file_url = 'https://{0}.file.core.windows.net/{1}'.format(
                cfg.storage_account_name, azure_file_share_name),
            relative_mount_path=azure_file_share)]
)

parameters = models.ClusterCreateParameters(
    location=cfg.location,
    vm_size='STANDARD_NC6',
    virtual_machine_configuration=models.VirtualMachineConfiguration(
        image_reference=models.ImageReference(
            publisher='microsoft-ads',
            offer='linux-data-science-vm-ubuntu',
            sku='linuxdsvmubuntu',
            version='latest')),
    scale_settings=models.ScaleSettings(
        manual=models.ManualScaleSettings(target_node_count=nodes_count)
    ),
    node_setup=models.NodeSetup(
        mount_volumes=volumes
    ),
    user_account_settings=models.UserAccountSettings(
        admin_user_name=cfg.admin,
        admin_user_password=cfg.admin_password,
        admin_user_ssh_public_key=cfg.admin_ssh_key
    )
)

### Create Compute Cluster

In [ ]:
_ = client.clusters.create(cfg.resource_group, cluster_name, parameters).result()

### Monitor Cluster Creation

Get the just created cluster. utilities.py contains a helper function to print out all kind of nodes count in the cluster.

In [ ]:
cluster = client.clusters.get(cfg.resource_group, cluster_name)
utilities.print_cluster_status(cluster)

### Deploy MNIST Dataset

For demonstration purposes, we will download preprocessed MNIST dataset to the current directory and upload it to file share directory named `mnist`.

In [ ]:
mnist_dataset_url = 'http://download.caffe2.ai/databases/mnist-lmdb.zip'
if not os.path.exists('mnist_train_lmdb') or not os.path.exists('mnist_test_lmdb'):
    utilities.download_file(mnist_dataset_url, 'caffe2_mnist_dataset.zip')
    print('Extracting Caffe2 MNIST dataset...')
    with zipfile.ZipFile('caffe2_mnist_dataset.zip', 'r') as z:
        z.extractall('.')
print('Done')

#### Create File Share and Upload MNIST Dataset

In [ ]:
mnist_dataset_directory = 'mnist'

There are multiple ways to create folders and upload files into Azure File Share - you can use [Azure Portal](https://ms.portal.azure.com), [Storage Explorer](http://storageexplorer.com/), [Azure CLI2](/azure-cli-extension) or Azure SDK for your preferable programming language.
In this example we will use Azure SDK for python to copy files into file share.

In [ ]:
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_directory(
    azure_file_share_name, mnist_dataset_directory,
    fail_on_exist=False)
# Since uploading can take significant time, let's check first if the
# file has been uploaded already.
for d in ['mnist-train-nchw-lmdb', 'mnist-test-nchw-lmdb']:
    service.create_directory(
        azure_file_share_name, os.path.join(mnist_dataset_directory, d),
        fail_on_exist=False)
    for f in ['data.mdb', 'lock.mdb']:
        if service.exists(azure_file_share_name, os.path.join(mnist_dataset_directory, d, f)):
            continue
        service.create_file_from_path(
            azure_file_share_name, os.path.join(mnist_dataset_directory, d), f, os.path.join(d,f))
print('Done')

### Deploy Sample Script and Configure the Input Directories

- For each job we will create a folder containing a copy of [resnet50_trainer.py](https://github.com/chainer/chainer/blob/master/examples/mnist/train_mnist.py). This allows each job to have it's own copy of the sample script (in case you would like to change it).

In [ ]:
caffe2SampleDir = 'Caffe2Samples'
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_directory(
    azure_file_share_name, caffe2SampleDir, fail_on_exist=False)
service.create_file_from_path(
    azure_file_share_name, caffe2SampleDir, 'resnet50_trainer.py', 'resnet50_trainer.py')

- The job needs to know where to find resnet50_trainer.py script. So, we will configure an input directory for the script:

In [ ]:
input_directories = [
    models.InputDirectory(
        id='SCRIPT',
        path='$AZ_BATCHAI_MOUNT_ROOT/{0}/{1}'.format(azure_file_share, caffe2SampleDir)),
    models.InputDirectory(
        id='DATASET',
        path='$AZ_BATCHAI_MOUNT_ROOT/{0}/{1}'.format(azure_file_share, mnist_dataset_directory))
]

The job will be able to reference those directories using ```$AZ_BATCHAI_INPUT_SCRIPT``` and ```$AZ_BATCHAI_INPUT_DATASET``` environment variables.

### Configure Output Directories
We will store standard and error output of the job in Azure File Share:

In [ ]:
std_output_path_prefix = '$AZ_BATCHAI_MOUNT_ROOT/{0}'.format(azure_file_share)

### Configure Job
- The job will use `caffe2ai/caffe2` container.
- Will use configured previously input and output directories;
- Will run modified `resnet50_trainer.py` from SCRIPT input directory;
- Will output standard output and error streams to file share;
- Will use $AZ_BATCHAI_SHARED_JOB_TEMP shared directory created by Batch AI to coordinate execution between nodes;
- For demostration purpose, we will only run 5 epochs with epoch size as 2000.


In [ ]:
parameters = models.job_create_parameters.JobCreateParameters(
     location=cfg.location,
     cluster=models.ResourceId(id=cluster.id),
     node_count=2,
     input_directories=input_directories,
     std_out_err_path_prefix=std_output_path_prefix,
     container_settings=models.ContainerSettings(
         image_source_registry=models.ImageSourceRegistry(image='caffe2ai/caffe2')),
     caffe2_settings = models.Caffe2Settings(
         python_script_file_path='$AZ_BATCHAI_INPUT_SCRIPT/resnet50_trainer.py',
         command_line_args='--num_shards 2 --shard_id $AZ_BATCHAI_TASK_INDEX --run_id 0 --epoch_size 2000 --num_epochs 5 --train_data $AZ_BATCHAI_INPUT_DATASET/mnist-train-nchw-lmdb --file_store_path $AZ_BATCHAI_SHARED_JOB_TEMP'))

### Create a training Job and wait for Job completion


In [ ]:
job_name = datetime.utcnow().strftime('caffe2_%m_%d_%Y_%H%M%S')
job = client.jobs.create(cfg.resource_group, job_name, parameters).result()
print('Created Job: {}'.format(job_name))

### Wait for Job to Finish
The job will start running when the cluster will have enough idle nodes. The following code waits for job to start running printing the cluster state. During job run, the code prints current content of stdout.txt.

**Note** Execution may take several minutes to complete.

In [ ]:
utilities.wait_for_job_completion(client, cfg.resource_group, job_name, cluster_name, 'stdouterr', 'stderr-0.txt')

### Download stdout.txt and stderr.txt files for the Job

In [ ]:
files = client.jobs.list_output_files(cfg.resource_group, job_name,
                                      models.JobsListOutputFilesOptions(outputdirectoryid='stdouterr')) 
for f in list(files):
    if f.download_url:
        utilities.download_file(f.download_url, f.name)
print('All files downloaded')

In [ ]:
for n in range(nodes_count):
    print('stdout-{0}.txt content:'.format(n))
    with open('stderr-{0}.txt'.format(n)) as f:
        print(f.read())

### Delete the Job

In [ ]:
_ = client.jobs.delete(cfg.resource_group, job_name)

### Delete the Cluster
When you are finished with the sample and don't want to submit any more jobs you can delete the cluster using the following code.

In [ ]:
_ = client.clusters.delete(cfg.resource_group, cluster_name)

### Delete File Share
When you are finished with the sample and don't want to submit any more jobs you can delete the file share completely with all files using the following code.

In [ ]:
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.delete_share(azure_file_share_name)